# [NTDS'18] milestone 3: spectral graph theory
[ntds'18]: https://github.com/mdeff/ntds_2018

[Michaël Defferrard](http://deff.ch), [EPFL LTS2](https://lts2.epfl.ch)

## Students

* Team: `<your team number>`
* Students: `<the name of all students in the team>`
* Dataset: `<the dataset you used to complete the milestone>`

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to two sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective

The goal of this milestone is to get familiar with the graph Laplacian and its spectral decomposition.

## 0 Load your network

In [164]:
%matplotlib inline

If you get a `No module named 'sklearn'` error when running the below cell, install [scikit-learn](https://scikit-learn.org) with `conda install scikit-learn` (after activating the `ntds_2018` environment).

In [165]:
import numpy as np
from scipy import sparse
import scipy.sparse.linalg
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [166]:
ADJACENCY_PATH = '../data/adjacency_matrices/'
ADJACENCY_COSINE_PATH = ADJACENCY_PATH + 'cosine'
ADJACENCY_EUC_PATH = ADJACENCY_PATH + 'eucledian'
MEMBER_ID_PATH = '../data/member_matrices/member_id_party'

Let's denote your graph as $\mathcal{G} = (\mathcal{V}, \mathcal{E}, A)$, where $\mathcal{V}$ is the set of nodes, $\mathcal{E}$ is the set of edges, $A \in \mathbb{R}^{N \times N}$ is the (weighted) adjacency matrix, and $N = |\mathcal{V}|$ is the number of nodes.

Import the adjacency matrix $A$ that you constructed in the first milestone.
(You're allowed to update it between milestones if you want to.)

In [167]:
adjacency =  np.load(ADJACENCY_COSINE_PATH+'.npy')
members_parties = np.load(MEMBER_ID_PATH+'.npy')
# Removing disconnected nodes
node_degrees = np.count_nonzero(adjacency, axis=1)
nodes_to_keep = np.nonzero(node_degrees)[0]
members_parties = members_parties[nodes_to_keep]
adjacency = adjacency[nodes_to_keep,:][:,nodes_to_keep]
n_nodes =  adjacency.shape[0]

## 1 Graph Laplacian

### Question 1

From the (weighted) adjacency matrix $A$, compute both the combinatorial (also called unnormalized) and the normalized graph Laplacian matrices.

Note: if your graph is weighted, use the weighted adjacency matrix. If not, use the binary adjacency matrix.

For efficient storage and computation, store these sparse matrices in a [compressed sparse row (CSR) format](https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_.28CSR.2C_CRS_or_Yale_format.29).

In [168]:
D= sparse.csr_matrix(np.diag(adjacency.sum(1)))
A = sparse.csr_matrix(adjacency)
laplacian_combinatorial = D - A
D_half = D.power(-0.5)
laplacian_normalized =  D_half.multiply(laplacian_combinatorial).multiply(D_half)

Use one of them as the graph Laplacian $L$ for the rest of the milestone.
We however encourage you to run the code with both to get a sense of the difference!

In [169]:
laplacian = laplacian_combinatorial
laplacian.shape

(92, 92)

### Question 2

Compute the eigendecomposition of the Laplacian $L = U^\top \Lambda U$, where the columns $u_k \in \mathbb{R}^N$ of $U = [u_1, \dots, u_N] \in \mathbb{R}^{N \times N}$ are the eigenvectors and the diagonal elements $\lambda_k = \Lambda_{kk}$ are the corresponding eigenvalues.

Make sure that the eigenvalues are ordered, i.e., $0 = \lambda_1 \leq \lambda_2 \leq \dots \leq \lambda_N$.

In [170]:
eigenvalues,eigenvectors =  scipy.linalg.eig(laplacian.todense())

eigenvectors = np.real(eigenvectors)
eigenvalues = np.real(eigenvalues)
indexes = np.argsort(eigenvalues)
eigenvalues = eigenvalues[indexes]
eigenvectors = eigenvectors[indexes]
assert eigenvectors.shape == (n_nodes, n_nodes)

In [171]:
eigenvalues

array([1.04848048e-15, 8.84561510e-15, 6.57234495e-01, 8.87519272e-01,
       2.63443947e+00, 3.89017791e+00, 4.59311500e+00, 6.31067867e+00,
       1.26454566e+01, 1.36408947e+01, 1.81130489e+01, 1.95034564e+01,
       2.12218926e+01, 2.25012666e+01, 2.25472999e+01, 2.45791406e+01,
       2.59247520e+01, 2.66669637e+01, 2.70647330e+01, 2.84090938e+01,
       2.92533114e+01, 2.93297356e+01, 2.96502383e+01, 3.02420235e+01,
       3.02824855e+01, 3.07442414e+01, 3.13052708e+01, 3.23301221e+01,
       3.24836890e+01, 3.25460722e+01, 3.30288308e+01, 3.30973946e+01,
       3.36995635e+01, 3.38117240e+01, 3.46276215e+01, 3.47778804e+01,
       3.50869130e+01, 3.54563560e+01, 3.54653248e+01, 3.56839383e+01,
       3.58189342e+01, 3.58710190e+01, 3.59519006e+01, 3.60861444e+01,
       3.61370150e+01, 3.62994971e+01, 3.64968294e+01, 3.65703635e+01,
       3.66345273e+01, 3.68857610e+01, 3.68898323e+01, 3.69518239e+01,
       3.70006969e+01, 3.70999308e+01, 3.71584912e+01, 3.72169600e+01,
      

In [172]:
eigenvectors.shape

(92, 92)

Justify your choice of eigensolver.

**Your answer here.**

### Question 3

We can write $L = S S^\top$. What is the matrix $S$? What does $S^\top x$, with $x \in \mathbb{R}^N$, compute?

**Your answer here.**

### Question 4

Show that $\lambda_k = \| S^\top u_k \|_2^2$, where $\| \cdot \|_2^2$ denotes the squared Euclidean norm (a.k.a. squared $L^2$ norm).

**Your answer here.**

What does the quantity $\| S^\top x \|_2^2$ tell us about $x$?

**Your answer here.**

### Question 5

What is the value of $u_0$, both for the combinatorial and normalized Laplacians?

**Your annswer here.**

### Question 6

Look at the spectrum of the Laplacian by plotting the eigenvalues.
Comment on what you observe.

In [173]:
# Your code here.

**Your answer here.**

How many connected components are there in your graph? Answer using the eigenvalues only.

In [174]:
# Your code here.

Is there an upper bound on the eigenvalues, i.e., what is the largest possible eigenvalue? Answer for both the combinatorial and normalized Laplacians.

**Your answer here.**

## 3 Laplacian eigenmaps

*Laplacian eigenmaps* is a method to embed a graph $\mathcal{G}$ in a $d$-dimensional Euclidean space.
That is, it associates a vector $z_i \in \mathbb{R}^d$ to every node $v_i \in \mathcal{V}$.
The graph $\mathcal{G}$ is thus embedded as $Z \in \mathbb{R}^{N \times d}$.

### Question 7

What do we use Laplacian eigenmaps for? (Or more generally, graph embeddings.)

**Your answer here.**

### Question 8

Embed your graph in $d=2$ dimensions with Laplacian eigenmaps.
Try with and without re-normalizing the eigenvectors by the degrees, then keep the one your prefer.

**Recompute** the eigenvectors you need with a partial eigendecomposition method for sparse matrices.
When $k \ll N$ eigenvectors are needed, partial eigendecompositions are much more efficient than complete eigendecompositions.
A partial eigendecomposition scales as $\Omega(k |\mathcal{E}|$), while a complete eigendecomposition costs $\mathcal{O}(N^3)$ operations.

In [175]:
# Your code here.

Plot the nodes embedded in 2D. Comment on what you see.

In [176]:
# Your code here.

**Your answer here.**

### Question 9

What does the embedding $Z \in \mathbb{R}^{N \times d}$ preserve?

**Your answer here.**

## 2 Spectral clustering

*Spectral clustering* is a method to partition a graph into distinct clusters.
The method associates a feature vector $z_i \in \mathbb{R}^d$ to every node $v_i \in \mathcal{V}$, then runs [$k$-means](https://en.wikipedia.org/wiki/K-means_clustering) in the embedding space $\mathbb{R}^d$ to assign each node $v_i \in \mathcal{V}$ to a cluster $c_j \in \mathcal{C}$, where $k = |\mathcal{C}|$ is the number of desired clusters.

### Question 10

Choose $k$ and $d$. How did you get to those numbers?

assuming that we have 2 distinct groups of republican and democrats, we should try k = 2 , or k = 3 to account for independent candidates. 

### Question 11

1. Embed your graph in $\mathbb{R}^d$ as $Z \in \mathbb{R}^{N \times d}$.
   Try with and without re-normalizing the eigenvectors by the degrees, then keep the one your prefer.
1. If you want $k=2$ clusters, partition with the Fiedler vector. For $k > 2$ clusters, run $k$-means on $Z$. Don't implement $k$-means, use the `KMeans` class imported from scikit-learn.

In [177]:
# Your code here.
def gen_kmeans(n_clusters,n_features,data,random_state):
    data = data[:,-1*n_features:]
    means = KMeans(n_clusters=n_clusters, random_state=0).fit(data)
    return means.labels_


### Question 12

Use the computed cluster assignment to reorder the adjacency matrix $A$.
What do you expect? What do you observe?

In [178]:
# Your code here.


**Your answer here.**

### Question 13

If you have ground truth clusters for your dataset, compare the cluster assignment from spectral clustering to the ground truth.
A simple quantitative measure is to compute the percentage of nodes that have been correctly categorized.
If you don't have a ground truth, qualitatively assess the quality of the clustering.

Ground truth clusters are the "real clusters".
For example, the genre of musical tracks in FMA, the category of Wikipedia articles, the spammer status of individuals, etc.
Look for the `labels` in the [dataset descriptions](https://github.com/mdeff/ntds_2018/tree/master/projects/README.md).

In [179]:
# Your code here.
def compare_truth(kmeans_labels,original_labels):
    #return accuracy, assign the first R to first label{0,1,2} in kmeans, find next unassigned value and assign it to 'D' 
    #rest to 'I'
    first_R_pos = np.where(original_labels == 'R')[0][0]
    first_D_pos = np.where(original_labels == 'D')[0][0]
    label_at_first_R_pos = kmeans_labels[first_R_pos]
    
    kmeans_l_copy = np.copy(kmeans_labels).astype(str)
    kmeans_l_copy[kmeans_labels == label_at_first_R_pos] = 'R'
    
    remaining = np.where(kmeans_l_copy[kmeans_l_copy != 'R' ])[0]
    label_at_first_D_pos = label_at_first_R_pos
    for i in remaining:
        if(not(kmeans_labels[i] == label_at_first_R_pos) and (original_labels[i] == 'D' )):
            label_at_first_D_pos = kmeans_labels[i]
            
    kmeans_l_copy[kmeans_labels == label_at_first_D_pos] = 'D'
    kmeans_l_copy[(kmeans_labels != label_at_first_R_pos) & (kmeans_labels != label_at_first_D_pos)] = 'I'
    return np.sum(kmeans_l_copy == original_labels.flatten())/len(kmeans_labels)
    

In [190]:


def get_best_seed(n_clusters,n_features,data,original_labels,n_tries):
    accs = []
    tries = np.random.randint(low=1, high=1e6, size=n_tries)
    for i  in tries:
        accs.append(compare_truth(gen_kmeans(n_clusters,n_features,data,i),original_labels))
    
    accs_ary = np.array(accs)
    index_max = np.argmax(accs_ary)
    return accs_ary[index_max],tries[index_max]
        
    
accuracies =[]
for i in range(1,92):
    accuracies.append(compare_truth(gen_kmeans(3,i,eigenvectors,0),members_parties))
accs_array = np.array(accuracies)
index_max = np.argmax(accs_array)
print('computing kmeans with eigenvectors,best accuracy {} for {} eigenvectors'.format(np.max(accs_array),index_max))
best_acc,best_seed = get_best_seed(3,93,adjacency,members_parties,100)
print('computing kmeans with original features,accuracy {} and best seed {}'.format(best_acc,best_seed))
print(*zip(gen_kmeans(3,93,adjacency,best_seed),members_parties.flatten()))

computing kmeans with eigenvectors,best accuracy 0.4891304347826087 for 6 eigenvectors
computing kmeans with original features,accuracy 0.8695652173913043 and best seed 72504
(1, 'R') (1, 'R') (2, 'D') (1, 'R') (2, 'D') (1, 'R') (1, 'R') (2, 'D') (2, 'D') (2, 'D') (2, 'D') (2, 'D') (2, 'D') (1, 'R') (0, 'R') (1, 'R') (1, 'R') (2, 'D') (1, 'R') (2, 'D') (1, 'R') (0, 'R') (2, 'D') (2, 'D') (2, 'D') (0, 'R') (2, 'D') (1, 'R') (1, 'R') (2, 'D') (0, 'R') (1, 'R') (1, 'R') (1, 'R') (0, 'D') (1, 'R') (1, 'R') (1, 'R') (2, 'D') (2, 'D') (1, 'R') (2, 'D') (2, 'D') (2, 'D') (1, 'R') (1, 'R') (1, 'R') (2, 'D') (2, 'I') (2, 'D') (1, 'R') (2, 'D') (1, 'R') (2, 'D') (1, 'R') (0, 'D') (1, 'R') (2, 'D') (0, 'R') (2, 'D') (2, 'D') (0, 'D') (2, 'D') (2, 'D') (1, 'R') (2, 'D') (1, 'R') (2, 'D') (1, 'R') (1, 'R') (1, 'R') (1, 'R') (0, 'I') (2, 'D') (1, 'R') (2, 'D') (2, 'D') (1, 'R') (2, 'D') (0, 'R') (1, 'R') (1, 'R') (2, 'D') (1, 'R') (2, 'D') (2, 'D') (1, 'R') (0, 'D') (2, 'D') (2, 'D') (0, 'D') (1, 'R

### Question 14

Plot the cluster assignment (one color per cluster) on the 2D embedding you computed above with Laplacian eigenmaps.

In [ ]:
# Your code here.

### Question 15

Why did we use the eigenvectors of the graph Laplacian as features? Could we use other features for clustering?

**Your answer here.**